#MIP Merge

This script merges the different data sets according to `b_crefo` (MUP ownership data, MUP MIP panel on the owners and the companies), turns them into panels and cleans the different columns

In [2]:
import pandas as pd
import numpy as np

Load the data into DataFrames

In [3]:
df_ownership = pd.read_csv(r"C:\Users\lucas\OneDrive\BA\Data\Ownership_Change\MUPOwn.csv", 
                           encoding="ISO-8859-1")
df_companies = pd.read_csv(r"C:\Users\lucas\OneDrive\BA\Data\Ownership_Change\MUPMIP_panel_owned.csv", 
                           encoding="ISO-8859-1")
df_owners = pd.read_csv(r"C:\Users\lucas\OneDrive\BA\Data\Ownership_Change\MUPMIP_panel_owner.csv", 
                        encoding="ISO-8859-1")

C:\Users\lucas\AppData\Local\Temp\ipykernel_23508\1245519110.py:3: DtypeWarning: Columns (3,13,32,33,34,35,36,37,39,47,58,59,60,61,62,63,64,65,68,69,72,73,77,82,85,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,183,184,190,192,193,194,195,197,201,202,203,204,215,219,220,221,222,223,224,231,232,235,236,238,239,240,241,242,244,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,370,384,392,393,394,395,396,397,411,413,419,422,423,424,425,426,427,428,4

The flag `b_is_main_owner` is used to seperate minority from majority shareholders (defined as the owner of at least 50% of the equity), and, where there is no information on the percentage owned, only owners with following "characteristics" (dt. Eigenschaft) were considered majority: "Owner" (Inhaber), "Shareholder" (Gesellschafter), "Limited Partner" (Kommanditist), "General Partner" (Komplementär), and "Majority Shareholder" (Hauptaktionär)

In [4]:
df_ownership["b_is_main_owner"] = np.where(df_ownership["b_anteil"] >= 50, True, 
                                           np.where((df_ownership["b_anteil"].isna())
                                           & (df_ownership["b_eigenschaft"].str.contains("Inhaber|Gesellschafter|Kommanditist|Komplementär|Hauptaktionär", regex=True)),
                                             True, False))

Specify the start and end year of the participation, as a step to turn the ownership data into a panel. The entries which have no start nor end date will be assigned all the years where there are observations in the MIP data set (first year 1993, last year 2021). The end date is set to 2023 for all participations which didn't end in the observation period or where there is no information so that 2021 is within the start to end range

In [5]:
df_ownership["b_start_year"] = df_ownership["b_beginn"].astype(str).str[:4]
df_ownership["b_end_year"] = df_ownership["b_ende"].astype(str).str[:4]
df_ownership["b_start_year"] = np.where(df_ownership["b_start_year"] == "0.0", 1993, df_ownership["b_start_year"])
df_ownership["b_start_year"] = np.where(df_ownership["b_start_year"] == "nan", 1993, df_ownership["b_start_year"])
df_ownership["b_end_year"] = np.where(df_ownership["b_end_year"] == "0.0", 2023, df_ownership["b_end_year"])
df_ownership["b_end_year"] = np.where(df_ownership["b_end_year"] == "nan", 2023, df_ownership["b_end_year"])

Parse `b_start_year` and `b_end_year` to integers

In [6]:
df_ownership["b_start_year"] = pd.to_numeric(df_ownership["b_start_year"], downcast="integer")
df_ownership["b_end_year"] = pd.to_numeric(df_ownership["b_end_year"], downcast="integer")

The array `survey_years` contains all the years with sample data for the companies in the MIP panel. Now I will create dummy variables for all sample years, so that we transform the ownership data frame later to a panel

In [7]:
survey_years = np.unique(df_companies["jahr"])
for i in range(len(survey_years)):
    df_ownership[str(survey_years[i])] = np.where((df_ownership["b_start_year"] <= survey_years[i]) 
                                                  & (df_ownership["b_end_year"] > survey_years[i]),True, False)

Drop `b_firma` since it the correct information is encoded in `b_person`. 

In [8]:
df_ownership.drop(labels="b_firma", axis=1, inplace=True)

Drop all the entries that are not a main owner

In [9]:
df_ownership = df_ownership[df_ownership["b_is_main_owner"] == True]
df_ownership.drop(labels=["b_is_main_owner"], axis=1, inplace=True)

If there are with the same `b_crefo` for the same year, keep the entry with the most recent wave

In [10]:
df_ownership.sort_values(by="welle", axis=0, ascending=False, inplace=True)
df_ownership.drop_duplicates(["b_crefo", "companyid"], keep="first", inplace=True)

Drop all companies where there are more than 5 owners, and then assign a number from 1 to 5 for the pivot table afterwards

In [11]:
df_ownership = df_ownership.groupby(["companyid"]).filter(lambda x: len(x) < 6)
df_ownership["owner_number"] = df_ownership.groupby(["companyid"])["b_crefo"].rank(method="min")

`df_panel_ownership` now has the structure of a panel data set, after using `df.melt()`

In [12]:
df_panel_ownership = df_ownership.melt(id_vars=['crefo', 'b_crefo', 'b_eigenschaft', 'b_betrag', 'b_anteil', 
                                                'b_beginn','b_ende', 'b_person', 'welle', 'companyid', 
                                                'ownerid', 'b_start_year', 'b_end_year', 'owner_number'],
                                                  var_name="panel_year")

In this step, I filtered the data set to only have the years where there was an observation of the owner (i.e., the owner was owned the company at that year), and drop the `value` variable afterwards. I also removed the owners which do not have a `companyid`.

In [13]:
df_panel_ownership = df_panel_ownership[df_panel_ownership["value"] == True]
df_panel_ownership = df_panel_ownership[df_panel_ownership["companyid"] >= 0]
df_panel_ownership.drop(labels=["value"], axis=1, inplace=True)

Create a pivot table so that with `b_crefo` as value to match afterwards with the ownership table. This enables us to later check for changes in ownership more easily. NaN are filled with zero so that ownership changes can be spotted by subtracting a year's `b_crefo` value by last year's value. 

In [14]:
df_pivot_ownership = df_panel_ownership.pivot(index=["companyid", "panel_year"], columns="owner_number", values="b_crefo").reset_index()
df_pivot_ownership.fillna(0, inplace=True)

Here we add up all the five different `b_crefo` together so that if there is a new owner or an old owner leaves, we can see that through the difference of this sum from one year to the next

In [15]:
df_pivot_ownership["owner_sum"] = df_pivot_ownership[1.0] + df_pivot_ownership[2.0] + df_pivot_ownership[3.0] + df_pivot_ownership[4.0] + df_pivot_ownership[5.0] 
df_pivot_ownership = df_pivot_ownership.assign(diff=df_pivot_ownership.groupby(['companyid']).owner_sum.diff())
df_pivot_ownership.fillna(0, inplace=True)
df_pivot_ownership["is_owner_change"] = np.where(df_pivot_ownership["diff"] != 0, True, False)                           

Left outer merge with companies data: the option `left` when merging the ownership panel data and the companies panel data ensures that all ownership panel entries are kept and match with the available company panel data


In [16]:
df_pivot_ownership["panel_year"] = pd.to_numeric(df_pivot_ownership["panel_year"])
df_merged_companies = pd.merge(df_pivot_ownership, df_companies, how="left", left_on=["panel_year", "companyid"], right_on=["jahr", "companyid"])

Create a ownerid table, where all `b_crefo` values are mapped to NaN or to a `ownerid` value

In [17]:
df_ownerid_table = pd.read_csv(r"C:\Users\lucas\OneDrive\BA\Data\Ownership_Change\MUPOwn.csv", 
                           encoding="ISO-8859-1")
df_ownerid_table = df_ownerid_table[["b_crefo", "ownerid"]]
df_ownerid_table.drop_duplicates(inplace=True)
df_ownerid_table.fillna(0, inplace=True)

Create columns with `ownerid` entries and rename them after the merger

In [18]:
df_merged_companies.rename({
        1.0 : "b_crefo_1",
        2.0 : "b_crefo_2",
        3.0 : "b_crefo_3",
        4.0 : "b_crefo_4",
        5.0 : "b_crefo_5"
    }, axis=1, inplace=True)
for i in np.arange(start=1, stop=6):
    current_column = "b_crefo_" + str(i)
    df_merged_companies = pd.merge(df_ownerid_table, df_merged_companies, left_on=["b_crefo"], right_on=[current_column], how="right", suffixes=(str(i), ''))

In [19]:
df_merged_companies.drop(["b_crefo5", "b_crefo4", "b_crefo3", "b_crefo2", "b_crefo"], axis=1, inplace=True)
df_merged_companies.rename({
    "ownerid" : "ownerid_1",
    "ownerid2" : "ownerid_2",
    "ownerid3" : "ownerid_3",
    "ownerid4" : "ownerid_4",
    "ownerid5" : "ownerid_5"
}, axis=1, inplace=True)

`is_ownerid_change` indicates if there is a change of ownership of owners which are included in the MIP data set (and therefore have an `ownerid`)

In [33]:
df_merged_companies["ownerid_sum"] = df_merged_companies["ownerid_1"] + df_merged_companies["ownerid_2"] +df_merged_companies["ownerid_3"] + df_merged_companies["ownerid_4"] + df_merged_companies["ownerid_5"]
df_merged_companies = df_merged_companies.assign(ownerid_diff=df_merged_companies.groupby(['companyid']).ownerid_sum.diff())
df_merged_companies["ownerid_diff"] = df_merged_companies["ownerid_diff"].fillna(0)
df_merged_companies["is_ownerid_change"] = np.where(df_merged_companies["ownerid_diff"] != 0, True, False)

Create a table where the `ownerid` are matched with their MIP panel data information 

In [ ]:
df_merged_owners = pd.merge(left=df_merged_companies, right=df_owners) #Resolve the problem of having more than one ownerid per line

Save the end result

In [37]:
df_merged_companies.to_csv(r"C:\Users\lucas\OneDrive\BA\Data\outputs\merged_companies.csv")